# WEAT (Word Embedding Association Test)

This notebook demonstrates recording a fairness experiment that follows [WEAT](https://arxiv.org/pdf/1608.07187) (Word Embedding Association Test), using [WEFE](https://github.com/dccuchile/wefe) (Word Embedding Fairness Evalution).

## A Brief Background

WEAT uses **cosine similarity** to measure the association. Generally speaking we would expect words of similar semantic meaning to occupy similar vector spaces, e.g. man, boy, guy etc. We would also expect word associations to be similarly distances from eachother, e.g. man - king should be roughly the same distance as women - queen.

In a fair word embedding space, we should expect the word “Professor” has similar distance to the words that represents different ethnic groups such as “Black”, “White” or “Asian”.  The similarity is:

**WEAT** compares sets of target words $X$ and $Y$ with sets of attributes $A$ and $B$ with the hypothesis that words in set $X$ are associated with words in set $A$ and likewise words in set $Y$ are associated with words in set $B$. For example:
$$
X = \{\text{man}, \text{male}, \text{boy}\}
$$
$$
Y = \{\text{woman}, \text{female}, \text{girl}\}
$$
$$
A = \{\text{doctor}, \text{professor}, \text{manager}\}
$$
$$
B = \{\text{nurse}, \text{teacher}, \text{assistant}\}
$$

and we hypothesise that our model may associate the words "male" with "professor" and "female" with "teacher" instead, as a measure of bias. Using cosine similarity for a word $\textbf{w}$ against words $\textbf{a},\textbf{b}$ in sets $A,B$ we have:

$$
s(\textbf{w}, A, B) = \frac{1}{n} \sum_{\textbf{a} \in A} \cos(\textbf{w}, \textbf{a}) - \frac{1}{n} \sum_{\textbf{b} \in B} \cos(\textbf{w}, \textbf{b})
$$

where $n$ is the number of embedded attributes (i.e. instances of a feature) in each set. To quantify the bias, we use the [effect size](https://en.wikipedia.org/wiki/Effect_size):
$$
d(X, Y, A, B) = \frac{\frac{1}{n} \sum_{x \in X} s(x, A, B) - \frac{1}{n} \sum_{y \in Y} s(y, A, B)}{\sigma_{w \in X \cup Y} s(w, A, B)}
$$

A positive effect size confirms that the words in set $X$ have some association with words in set $A$ and likewise words in $Y$ are associated with words in $B$.

To determine the statistical significance of biases measured in these tests, the authors use the statistic:
$$
s(X,Y,A,B) = \sum_{x \in X} s(x,A,B) - \sum_{y \in Y} s(y,A,B)
$$

for a permutation test with partitions $(X_i,Y_i)$ of $X \cup Y$:
$$
p = P_r[s(X_i,Y_i,A,B) > s(X,Y,A,B)]
$$

## Import the Libraries and Embedding Model

In this tutorial, we are using [GloVe](https://nlp.stanford.edu/projects/glove/) vectors, a model which generated word embeddings trained on different corpuses (datasets). In other words, we download the actual word embeddings, the representation of words in a vector space.

In [ ]:
# Imports
import gensim.downloader as api

from wefe.datasets import load_weat
from wefe.metrics import WEAT, MAC
from wefe.query import Query
from wefe.word_embedding_model import WordEmbeddingModel

import pandas as pd
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import seaborn as sns

glove_twitter_50= api.load("glove-twitter-50")
# WordEmbeddingModel receives as first argument a KeyedVectors model
# and the second argument the model name.
model = WordEmbeddingModel(glove_twitter_50, "glove twitter dim=50")

[==================================================] 100.0% 199.5/199.5MB downloaded


**Metadata Checklist:**
- [ ] SBOM, AIBOM is available and updated (Check how you can create them: )
- [ ] Glove Model Card is synced and updated (Check how to sync model card from popular providers)

WEFE uses a "Query" structure, containing target and attribute sets. Returning to the example above, using sets $X$ and $Y$ containing terms relating to males and females, and sets $A$ and $B$ containing terms relating to profession, we can create our query as follows, expecting a positive effect size, which would indicate the terms in set $A$ are potentially stereotypical of words in set $X$ and similarily with $B$ and $Y$.

In [ ]:
query = Query(
    target_sets=[
        ["male", "man", "boy"],
        ["female", "woman", "girl"],
    ],
    attribute_sets=[
        [
            "doctor",
            "professor",
            "manager",
        ],
        [
            "nurse",
            "teacher",
            "assistant"
        ],
    ],
    target_sets_names=["male-terms", "female-terms"],
    attribute_sets_names=["A", "B"],
)

print(query)

<Query: male-terms and female-terms wrt A and B
- Target sets: [['male', 'man', 'boy'], ['female', 'woman', 'girl']]
- Attribute sets:[['doctor', 'professor', 'manager'], ['nurse', 'teacher', 'assistant']]>


The words in the query are then obtained from the word vectors and the effect size is calculated. In this example, we see a positive effect size of $~1.46$, confirming the hypothesis that male terms are stereotypical with the words in attribute set $A$ and likewise with female terms and set $B$.

In [ ]:
metric = WEAT()
result = metric.run_query(query,
                          model,
                          preprocessors=[{"lowercase": True, "strip_accents": True}]
                          )
print(result)

{'query_name': 'male-terms and female-terms wrt A and B', 'result': 0.23768330613772082, 'weat': 0.23768330613772082, 'effect_size': 1.4585827575036083, 'p_value': nan}


We can extend this idea to terms commonly used in the financial domain. For example, certain countries will have certain stereotypes. We hypothesise that western countries, such as the USA, UK and Germany may be associated with the terms "democracy, polite and engineering" and eastern/non-western countries such as India, China and Russia may be stereotyped with words such as "Poverty, communism and revolution".

(note these stereotypical terms were generated by ChatGPT)

In [ ]:
query = Query(
    target_sets=[
        ["usa", "uk", "germany"],
        ["india", "china", "russia"],
    ],
    attribute_sets=[
        [
            "democracy",
            "polite",
            "engineering",
        ],
        [
            "poverty",
            "communism",
            "revolution"
        ],
    ],
    target_sets_names=["west-terms", "east-terms"],
    attribute_sets_names=["A", "B"],
)

print(query)

<Query: west-terms and east-terms wrt A and B
- Target sets: [['usa', 'uk', 'germany'], ['india', 'china', 'russia']]
- Attribute sets:[['democracy', 'polite', 'engineering'], ['poverty', 'communism', 'revolution']]>


In [ ]:
metric = WEAT()
result = metric.run_query(query,
                          model,
                          preprocessors=[{"lowercase": True, "strip_accents": True}]
                          )
print(result)

{'query_name': 'west-terms and east-terms wrt A and B', 'result': 0.14938854674498248, 'weat': 0.14938854674498248, 'effect_size': 1.5792070363528818, 'p_value': nan}


There are also other word embedding bias evaluation metrics we can use, for example MAC (Mean Average Cosine similarity) introduced by [Manzini et al](https://arxiv.org/abs/1904.04047). It takes a single word $t$ and computes its bias towards a set of attributes $A$ using the cosine distance as the reciprocal of the cosine similarity:

$$
S(t,A) = \frac{1}{N} \sum_{a \in A} 1 - \cos(t,a)
$$

Here we only have target words in the set $T$ compared to two with WEAT, $X$ and $Y$. The bias of the words in the set $T$ towards the words in the set $A$ is given by:

$$
\text{MAC}(T,A) = \frac{1}{|T||A|} \sum_{t \in T} \sum_{a \in A} S(t,A)
$$

In [ ]:
metric = MAC()
result = metric.run_query(query,
                          model,
                          preprocessors=[{"lowercase": True, "strip_accents": True}]
                          )
print(result)

{'query_name': 'west-terms and east-terms wrt A and B', 'result': 0.5728942226204606, 'mac': 0.5728942226204606, 'targets_eval': {'west-terms': {'usa': {'A': 0.7031929343938828, 'B': 0.6716323544581732}, 'uk': {'A': 0.5324387947718302, 'B': 0.49493031700452167}, 'germany': {'A': 0.6271928548812866, 'B': 0.540922741095225}}, 'east-terms': {'india': {'A': 0.5479872971773148, 'B': 0.4469280540943146}, 'china': {'A': 0.6379668464263281, 'B': 0.5510289669036865}, 'russia': {'A': 0.6186199535926183, 'B': 0.501889556646347}}}}


Now, let's save these results in the fairness log and then summarise the fairness test as part of the global-level model card information.